In [1]:
import pandas
import requests
import datetime
import time
import json

In [2]:
etherscan_api_key = "SU775DFF8GEGMER3B8ZPK3QZIJWGIBE45T"
url_endpoint = "https://api.etherscan.io/api"

In [6]:
#Starting Point for analysis
starting_block = 4826779 #Dec 31st 2017 start
ending_block = 4832685 #Dec 31st 2017 end
hex_starting_block = hex(starting_block)
hex_end_block = hex(ending_block)
get_blocks_by_range = 1

In [7]:
wei_in_ETH = 1000000000000000000

def tx_cost_in_ETH(gasused, gasprice):
    '''
    Does the conversion to get the actual transaction fee (in ETH) for each ethereum transaction.
    '''
    cost_in_eth = (gasused * gasprice)/float(wei_in_ETH)
    return cost_in_eth

In [8]:
#Datapoints per transaction needed
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data (I don't see this here... I'll skip it for now)
# Network Name (duh)
# Transaction Status
# Value of Tx

In [9]:
#Transactions processed for this group of blocks
processed_txs = []

#Loop for all transactions
for b in range(get_blocks_by_range):
    #Get Transactions in a Block
    #https://github.com/ethereum/wiki/wiki/JSON-RPC#eth_gettransactionbyhash
    next_block = starting_block + b
    hex_next_block = hex(next_block)
    
    if b%10 == 0:
        print "Now on Block: " + str(next_block)
        print "Total Block Count: " + str(b)
    
    #Go and grab all transactions in the block
    params = {"module":"proxy", "action":"eth_getBlockByNumber", "tag" : hex_starting_block, "boolean":"true", "apikey": etherscan_api_key}
    resp = requests.get(url_endpoint, params=params)
    resp_dict = json.loads(resp.content) #Get a string, must eval to a dict

    #Timestamp

    timestamp = int(resp_dict['result']['timestamp'],0)
    unix_timestamp = int(resp_dict['result']['timestamp'],0)
    timestamp = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")

    #Every other feature will be found in the transactions

    transactions = resp_dict['result']['transactions']

    for i in transactions:
        #Initialize the next transaction
        next_tx = {'datetime': timestamp, 'unixtimestamp': unix_timestamp, 'network': "Ethereum"}

        #Unique Hash
        next_tx['txhash'] = i['hash']

        #Account Sending
        next_tx['sender'] = i['from']

        #Account Receiving
        next_tx['receiver'] = i['to']

        #Tx Fee
        #Can't evaluate the fee until we get the receipt info, which is a separate call.
        #Oh Bother.
        next_tx['gasprice'] = int(i['gasPrice'],0)  
        
        #Value
        #converting from hex, to int, then from wei to ETH
        next_tx['value'] = int(i['value'],0)/float(wei_in_ETH)

        processed_txs.append(next_tx)

#Now I'm going through and grabbing the transaction hashes

total_transactions_to_process = len(processed_txs)

print "Total count of txs: " + str(total_transactions_to_process)
print datetime.datetime.now()

for n, i in enumerate(processed_txs):
    tx_hash = i['txhash']
    
    if n%100 == 0:
        print "Total Transactions processed: " + str(n)
        print datetime.datetime.now()

    params = {"module":"proxy", "action":"eth_getTransactionReceipt", "txhash" : tx_hash, "apikey": etherscan_api_key}
    resp = requests.get(url_endpoint, params=params)
    resp_dict = json.loads(resp.content)

    i["gasused"] = int(resp_dict['result']['gasUsed'],0)
    i['status'] = int(resp_dict['result']['status'],0)
    i['txfee'] = tx_cost_in_ETH(i['gasused'], i['gasprice'])

print "Completed"
print datetime.datetime.now()

Now on Block: 4826779
Total Block Count: 0
Total count of txs: 54
2018-01-09 11:25:28.953000
Total Transactions processed: 0
2018-01-09 11:25:28.957000
Completed
2018-01-09 11:25:52.390000


In [12]:
transactions[2]

{u'blockHash': u'0x014922ec09e6886719ec433e9ce3d94d93ae1dad582bc15d5995e1fe034237a0',
 u'blockNumber': u'0x49a69b',
 u'from': u'0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be',
 u'gas': u'0x13e96',
 u'gasPrice': u'0xcce416600',
 u'hash': u'0x7ddea9cbd09ae5c4044f6f04c36b047d046f1fb2d9761aa4dcde1d49abefac38',
 u'input': u'0xa9059cbb0000000000000000000000007a7d4ab95aceb2f02e2cb5f375b47cb2b432ed750000000000000000000000000000000000000000000000007a1fe16027700000',
 u'nonce': u'0xfda37',
 u'r': u'0x6846d4000a3d910fcbf811bf25259a8d6ca8e194ba0b5446dae6f56e25a8ec72',
 u's': u'0x4a8c4312715afd83f0dde01153d69249415e1bcf5daa2d144b89f9f37c96bdfc',
 u'to': u'0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0',
 u'transactionIndex': u'0x2',
 u'v': u'0x25',
 u'value': u'0x0'}

In [39]:
df = pandas.DataFrame.from_dict(processed_txs)

df.to_csv("Ethereum_20171231_transactions.csv")